In [2]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)



def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f


def cyclotomic_factor(n):
    """
    return the cyclotmic factor of F_{3p}
    """
    p = n//3 
    divisor_set = set()
    for d in range(2,p):
        if (p-1)%d ==0 and d!=3:
            divisor_set.add(d)
    if p % 4==1:
        divisor_set.add(8)
    cyc_factor = x
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor

def fekete(n):
    cyc_factor = cyclotomic_factor(n)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 
    
        
def reduced_fekete(d):
    f=fekete(d)
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        

def reduced_fekete_reduction(g,q):
    g=g.change_ring(GF(q))
    return g.factor()

def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 

def search_two_four_cycle_2(f,m,n):
    for q in range(m,n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 



def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result

In [19]:
f=fekete(3*5)
f

x^6 - x^4 + x^3 - x^2 + 1

In [20]:
g=reduced_fekete(3*5)
g

x^3 - 4*x + 1

In [7]:
fekete_reduction(g, 7)

(x^2 + x + 4) * (x^3 + 4) * (x^3 + 2*x + 1)

In [8]:
f=fekete(3*7)
fekete_reduction(f,227)

(x^2 + 12*x + 1) * (x^7 + 78*x^6 + 173*x^5 + 18*x^4 + 119*x^3 + 129*x^2 + 107*x + 9) * (x^7 + 138*x^6 + 90*x^5 + 215*x^4 + 2*x^3 + 221*x^2 + 160*x + 101)

# Galois group of $g_n$ where $n=3p$ and $p < 1000$

In [26]:
P=Primes()
n=10**5
p=3
while p < 100: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

5 [3, 2, 2]
7 [5, 19, 7]
11 [3, 5, 41]
13 [29, 43, 71]
17 [23, 199, 29]
19 [197, 373, 7]
23 [3, 41, 107]
29 [43, 181, 19]
31 [107, 23, 263]
37 [19, 179, 67]
41 [13, 5, 19]
43 [73, 257, 137]
47 [157, 547, 13]
53 [31, 199, 37]
59 [373, 103, 73]
61 [751, 367, 53]
67 [127, 73, 5]
71 [43, 1553, 101]
73 [173, 29, 13]
79 [311, 101, 107]
83 [307, 37, 541]
89 [919, 1229, 281]
97 [367, 149, 199]
101 [19, 317, 17]


In [27]:
P=Primes()
n=10**5
p=101
while p < 300: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

103 [101, 1571, 89]
107 [401, 509, 47]
109 [379, 211, 13]
113 [193, 1847, 29]
127 [5, 109, 53]
131 [3, 367, 109]
137 [157, 89, 211]
139 [13, 2971, 109]
149 [1033, 709, 461]
151 [67, 509, 137]
157 [1373, 83, 353]
163 [829, 4397, 83]
167 [739, 1567, 193]
173 [3, 3359, 29]
179 [331, 643, 109]
181 [17, 197, 199]
191 [613, 2731, 41]
193 [967, 743, 359]
197 [389, 349, 227]
199 [1657, 1373, 13]
211 [431, 167, 263]
223 [97, 733, 1777]
227 [229, 83, 107]
229 [1627, 4133, 467]
233 [3631, 1559, 89]
239 [433, 4957, 41]
241 [2417, 401, 317]
251 [1549, 643, 19]
257 [157, 2909, 137]
263 [1249, 131, 313]
269 [3371, 563, 281]
271 [5, 1973, 11]
277 [1117, 269, 467]
281 [2459, 1097, 31]
283 [3719, 2731, 317]
293 [2083, 479, 109]
307 [1231, 233, 5]


In [28]:
P=Primes()
n=10**5
p=307
while p < 400: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

311 [1223, 1553, 37]
313 [2389, 1597, 67]
317 [103, 3433, 181]
331 [5813, 401, 19]
337 [881, 557, 7]
347 [239, 2309, 269]
349 [19571, 5653, 331]
353 [2399, 2857, 227]
359 [419, 4373, 41]
367 [131, 3079, 83]
373 [17519, 2039, 19]
379 [113, 2203, 389]
383 [1367, 1303, 751]
389 [7237, 773, 223]
397 [89, 1129, 131]
401 [2879, 601, 557]


In [36]:
%%time
P=Primes()
n=10**5
p=401
while p < 450: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

409 [1453, 997, 293]
419 [127, 1297, 13]
421 [6449, 3407, 103]
431 [433, 1607, 41]
433 [3701, 2647, 173]
439 [31, 577, 7]
443 [503, 1493, 11]
449 [4733, 8581, 241]
457 [2957, 1259, 593]
CPU times: user 3min 33s, sys: 353 ms, total: 3min 33s
Wall time: 3min 45s


In [37]:
%%time
P=Primes()
n=10**5
p=457
while p < 500: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

461 [1049, 3851, 211]
463 [2699, 2417, 313]
467 [2729, 131, 113]
479 [1787, 2447, 313]
487 [389, 4549, 7]
491 [13691, 1153, 929]
499 [227, 197, 5]
503 [17, 1877, 2411]
CPU times: user 4min 16s, sys: 278 ms, total: 4min 17s
Wall time: 4min 21s


In [38]:
%%time
P=Primes()
n=10**5
p=503
while p < 600: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

509 [3, 17389, 859]
521 [2237, 863, 349]
523 [1973, 2843, 109]
541 [787, 1567, 37]
547 [2939, 457, 241]
557 [2113, 6299, 443]
563 [15629, 727, 241]
569 [7757, 3919, 23]
571 [10831, 1061, 2017]
577 [571, 503, 7]
587 [727, 1021, 157]
593 [1103, 8039, 109]
599 [4787, 7393, 47]
601 [11117, 503, 349]
CPU times: user 16min 34s, sys: 1.55 s, total: 16min 36s
Wall time: 16min 50s


In [39]:
%%time
P=Primes()
n=10**5
p=601
while p < 700: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

607 [11149, 11351, 419]
613 [659, 2797, 211]
617 [2131, 6079, 151]
619 [61, 4229, 5]
631 [3613, 4903, 43]
641 [6907, 577, 463]
643 [101, 7717, 61]
647 [3, 433, 11]
653 [2017, 1811, 67]
659 [2137, 10597, 631]
661 [12107, 1973, 127]
673 [2543, 103, 101]
677 [3037, 4259, 587]
683 [883, 131, 89]
691 [2029, 1747, 83]
701 [2819, 743, 47]
CPU times: user 20min 51s, sys: 1.94 s, total: 20min 53s
Wall time: 21min 40s


In [3]:
%%time
P=Primes()
n=10**5
p=701
while p < 750: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

709 [17389, 11497, 1187]
719 [641, 17351, 47]
727 [401, 32839, 1031]
733 [2473, 8629, 251]
739 [13, 151, 463]
743 [101, 7829, 83]
751 [6563, 4999, 491]
CPU times: user 26min 23s, sys: 1.46 s, total: 26min 25s
Wall time: 26min 51s


In [4]:
%%time
P=Primes()
n=10**5
p=751
while p < 780: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

757 [27107, 6529, 193]
761 [3413, 373, 89]
769 [12263, 4001, 31]
773 [8681, 12979, 359]
787 [2251, 7643, 1193]
CPU times: user 22min 22s, sys: 1.21 s, total: 22min 23s
Wall time: 22min 44s


In [5]:
%%time
P=Primes()
n=10**5
p=787
while p < 810: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

797 [6547, 6053, 193]
809 [419, 5351, 101]
811 [2083, 8093, 317]
CPU times: user 8min 12s, sys: 386 ms, total: 8min 13s
Wall time: 8min 17s


In [6]:
%%time
P=Primes()
n=10**5
p=811
while p < 850: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

821 [10771, 569, 521]
823 [3709, 15937, 479]
827 [1103, 751, 281]
829 [2699, 337, 467]


In [4]:
%%time
P=Primes()
n=10**5
p=829
while p < 840: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

839 [46457, 2657, 587]
853 [2069, 21871, 103]
CPU times: user 24min 25s, sys: 4.01 s, total: 24min 30s
Wall time: 25min 23s


In [5]:
%%time
P=Primes()
n=10**5
p=853
while p < 870: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

857 [2137, 491, 113]
859 [1051, 8609, 347]
863 [4733, 647, 809]
877 [3761, 151, 659]
CPU times: user 8min 7s, sys: 1.12 s, total: 8min 8s
Wall time: 8min 18s


In [6]:
%%time
P=Primes()
n=10**5
p=877
while p < 890: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

881 [71, 347, 37]
883 [3907, 2423, 241]
887 [13297, 3253, 2609]
907 [1061, 4057, 367]
CPU times: user 11min 36s, sys: 1.49 s, total: 11min 38s
Wall time: 11min 52s


In [3]:
%%time
P=Primes()
n=10**5
p=907
while p < 920: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

911 [33161, 5851, 313]
919 [4201, 5839, 379]
929 [397, 9733, 1607]
CPU times: user 23min 47s, sys: 2.33 s, total: 23min 49s
Wall time: 24min 8s


In [4]:
%%time
P=Primes()
n=10**5
p=929
while p < 950: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

937 [15053, 27697, 701]
941 [5347, 1327, 19]
947 [33703, 46769, 131]
953 [757, 6679, 601]
CPU times: user 57min 18s, sys: 5.36 s, total: 57min 23s
Wall time: 58min 14s


In [4]:
%%time
P=Primes()
n=10**5
p=953
while p < 980: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

967 [6571, 6857, 19]
971 [2663, 1657, 821]
977 [3719, 607, 107]


In [3]:
%%time
P=Primes()
n=10**5
p=977
while p < 990: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

983 [2551, 6829, 101]
991 [11971, 3389, 281]
CPU times: user 11min 8s, sys: 928 ms, total: 11min 9s
Wall time: 11min 20s


In [4]:
%%time
P=Primes()
n=10**5
p=991
while p < 1000: 
    p=P.next(p)
    g = reduced_fekete(3*p)
    print(p, triple(g,n))

997 [24197, 3739, 83]
1009 [2131, 10559, 137]
CPU times: user 18min 39s, sys: 1.24 s, total: 18min 41s
Wall time: 18min 54s


# Galois group of $f_{3p}$

First, we consider the case $p \equiv 1 \pmod{3}$. In this case, the discriminant of $f_{3p}$ is not a perfect square. To show that the Galois group of $f_{3p}$ is $(\Z/2)^n \rtimes S_n$ where $\deg(f_{3p})=2n$, we need to show that the Galois group of $f_{3p}$ contains a 2-cycle. The following codes find the smallest prime $q$ such that the reduction of $f_{3p}$ modulo $q$ produces a 2-cycle. 

In [5]:
P=Primes()
n=10**5
p=3
while p < 100: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

7 227
13 557
19 233
31 641
37 1049
43 293
61 11
67 1871
73 2699
79 1217
97 1091


In [6]:
P=Primes()
n=10**5
p=97
while p < 300: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

103 1109
109 929
127 1163
139 401
151 509
157 1061
163 2579
181 197
193 11027
199 2357
211 9551
223 2333
229 557
241 1163
271 311
277 269
283 2579
307 1223


In [7]:
P=Primes()
n=10**5
p=307
while p < 400: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

313 2591
331 821
337 557
349 881
367 7757
373 2039
379 3911
397 7817


In [8]:
P=Primes()
n=10**5
p=397
while p < 450: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

409 827
421 1559
433 6323
439 4751
457 13163


In [4]:
P=Primes()
n=10**5
p=457
while p < 500: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

463 227


In [7]:
P=Primes()
n=10**5
p=463
while p < 500: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

487 8147
499 1289


In [5]:
P=Primes()
n=10**5
p=499
while p < 530: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

523 2039
541 8243


In [6]:
P=Primes()
n=10**5
p=541
while p < 600: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

547 719
571 1061
577 503
601 1223


In [3]:
P=Primes()
n=10**5
p=601
while p < 650: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

607 9491
613 12347
619 4229
631 17327
643 2531


In [6]:
P=Primes()
n=10**5
p=643
while p < 680: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

661 257
673 3539


In [3]:
P=Primes()
m=18000
n=20000
p=673
while p < 690: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle_2(f,m,n))

691 18743


In [4]:
P=Primes()
n=10**5
p=691
while p < 720: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

709 3923
727 137


In [2]:
P=Primes()
n=10**5
p=727
while p < 740: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

733 13619


In [2]:
P=Primes()
n=10**5
p=733
while p < 750: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

739 15233
751 1283


In [3]:
P=Primes()
n=10**5
p=751
while p < 780: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

757 5261


In [2]:
P=Primes()
n=10**5
p=757
while p < 770: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

769 4673


In [3]:
P=Primes()
n=10**5
p=769
while p < 780: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

787 2141


In [6]:
P=Primes()
n=10**5
p=787
while p < 810: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

811 8627


In [7]:
P=Primes()
n=10**5
p=811
while p < 830: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

823 22367
829 773


In [8]:
P=Primes()
n=10**5
p=829
while p < 840: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

853 14549


In [2]:
P=Primes()
n=10**5
p=853
while p < 860: 
    p=P.next(p)
    if p % 3 ==1: 
        f = fekete(3*p)
        print(p, two_cycle(f,n))

859 8609


In [3]:
%%time 
m= 1000
n= m+2000
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

877 -1
CPU times: user 14min 18s, sys: 1.27 s, total: 14min 19s
Wall time: 14min 35s


In [3]:
%%time 
m= 3000
n= m+2000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

5000
877 -1
CPU times: user 11min 6s, sys: 1.29 s, total: 11min 7s
Wall time: 11min 18s


In [4]:
%%time 
m= 5000
n= m+2000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

7000
877 -1
CPU times: user 16min 58s, sys: 1.59 s, total: 17min
Wall time: 17min 21s


In [5]:
%%time 
m= 7000
n= m+2000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

9000
877 -1
CPU times: user 14min 51s, sys: 2.5 s, total: 14min 53s
Wall time: 15min 16s


In [6]:
%%time 
m= 9000
n= m+2000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

11000
877 -1
CPU times: user 13min 52s, sys: 1.59 s, total: 13min 53s
Wall time: 14min 7s


In [7]:
%%time 
m= 11000
n= m+3000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

14000
877 -1
CPU times: user 22min, sys: 4.27 s, total: 22min 4s
Wall time: 22min 40s


In [8]:
%%time 
m= 14000
n= m+3000
print(n)
p=877
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

17000
877 14783
CPU times: user 5min 10s, sys: 735 ms, total: 5min 11s
Wall time: 5min 17s


In [11]:
p = 877
P=Primes()
result = []
while p<1000:
    p=P.next(p)
    if p % 3 ==1:
        result.append(p)
print(result)    

[883, 907, 919, 937, 967, 991, 997, 1009]


In [12]:
%%time 
m= 0
n= m+3000
print(n)
p=883
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
883 2423
CPU times: user 14min 20s, sys: 2.49 s, total: 14min 23s
Wall time: 14min 44s


In [13]:
%%time 
m= 0
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
907 -1
CPU times: user 18min 29s, sys: 3.67 s, total: 18min 33s
Wall time: 19min 3s


In [14]:
%%time 
m= 3000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

6000
907 -1
CPU times: user 22min 26s, sys: 3.45 s, total: 22min 29s
Wall time: 23min 4s


In [15]:
%%time 
m= 6000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

9000
907 -1
CPU times: user 22min 58s, sys: 3.22 s, total: 23min 1s
Wall time: 23min 38s


In [16]:
%%time 
m= 9000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

12000
907 -1
CPU times: user 26min 16s, sys: 4.64 s, total: 26min 20s
Wall time: 26min 56s


In [17]:
%%time 
m= 12000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

15000
907 -1
CPU times: user 23min 45s, sys: 4.57 s, total: 23min 49s
Wall time: 24min 19s


In [18]:
%%time 
m= 15000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

18000
907 -1
CPU times: user 27min 3s, sys: 4.84 s, total: 27min 8s
Wall time: 28min 5s


In [19]:
%%time 
m= 18000
n= m+3000
print(n)
p=907
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

21000
907 20327
CPU times: user 21min 48s, sys: 3.03 s, total: 21min 51s
Wall time: 22min 45s


In [20]:
%%time 
m= 0
n= m+3000
print(n)
p=919
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
919 -1
CPU times: user 21min 39s, sys: 2.88 s, total: 21min 42s
Wall time: 22min 19s


In [21]:
%%time 
m= 3000
n= m+3000
print(n)
p=919
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

6000
919 3491
CPU times: user 2min 49s, sys: 512 ms, total: 2min 49s
Wall time: 2min 53s


In [2]:
%%time 
m= 0
n= m+3000
print(n)
p=937
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
937 509
CPU times: user 3min 12s, sys: 441 ms, total: 3min 13s
Wall time: 3min 17s


In [3]:
%%time 
m= 0
n= m+3000
print(n)
p=967
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
967 -1
CPU times: user 24min 12s, sys: 4.56 s, total: 24min 16s
Wall time: 25min 10s


In [5]:
%%time 
m= 3000
n= m+3000
print(n)
p=967
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

6000
967 -1
CPU times: user 24min 19s, sys: 2.6 s, total: 24min 21s
Wall time: 25min 27s


In [6]:
%%time 
m= 6000
n= m+3000
print(n)
p=967
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

9000
967 6857
CPU times: user 7min 20s, sys: 502 ms, total: 7min 21s
Wall time: 7min 27s


In [7]:
%%time 
m= 0
n= m+3000
print(n)
p=991
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
991 1091
CPU times: user 8min 14s, sys: 533 ms, total: 8min 15s
Wall time: 8min 22s


In [8]:
%%time 
m= 0
n= m+3000
print(n)
p=997
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

3000
997 -1
CPU times: user 25min 21s, sys: 1.87 s, total: 25min 23s
Wall time: 26min 12s


In [10]:
%%time 
m= 3000
n= m+3000
print(n)
p=997
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

6000
997 -1
CPU times: user 30min 51s, sys: 4.3 s, total: 30min 55s
Wall time: 31min 30s


In [11]:
%%time 
m= 6000
n= m+3000
print(n)
p=997
f = fekete(3*p)
print(p, two_cycle_2(f,m,n))

9000
997 7673
CPU times: user 15min 9s, sys: 1.44 s, total: 15min 11s
Wall time: 15min 29s


# Galois group of $f_{3p}$ when $p \equiv 2 \pmod{3}$

In this case, the discrimiant of $f_{3p}$ is a perfect square. To verify that it is maximal, we need to find a 2-4 cycle. 

In [12]:
P=Primes()
n=10**5
p=3
while p < 100: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

5 2
11 193
17 29
23 257
29 953
41 263
47 13
53 1237
59 719
71 2207
83 877
89 281
101 4519


In [13]:
%%time
P=Primes()
n=10**5
p=101
while p < 200: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

107 3079
113 1009
131 10531
137 2677
149 4783
167 3733
173 7607
179 5801
191 13309
197 4451
CPU times: user 8min 14s, sys: 1.54 s, total: 8min 15s
Wall time: 8min 49s


In [4]:
%%time
P=Primes()
n=10**5
p=197
while p < 250: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

227 7321
233 89


In [3]:
%%time
P=Primes()
n=10**5
p=233
while p < 250: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

239 27791
251 5167
CPU times: user 12min 6s, sys: 1.58 s, total: 12min 8s
Wall time: 12min 26s


In [4]:
%%time
P=Primes()
n=10**5
p=251
while p < 270: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

257 3461
263 1997
269 281
CPU times: user 2min 2s, sys: 282 ms, total: 2min 2s
Wall time: 2min 9s


In [5]:
%%time
P=Primes()
n=10**5
p=269
while p < 280: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

281 11657
CPU times: user 5min 55s, sys: 1.14 s, total: 5min 56s
Wall time: 6min 2s


In [3]:
%%time
P=Primes()
n=10**5
p=281
while p < 300: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

293 17053
CPU times: user 9min 28s, sys: 1.56 s, total: 9min 30s
Wall time: 9min 41s


In [4]:
%%time
P=Primes()
n=10**5
p=293
while p < 310: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

311 37
CPU times: user 1.15 s, sys: 7.95 ms, total: 1.15 s
Wall time: 1.26 s


In [5]:
%%time
P=Primes()
n=10**5
p=311
while p < 320: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

317 5081
CPU times: user 3min 2s, sys: 278 ms, total: 3min 3s
Wall time: 3min 12s


In [7]:
%%time
P=Primes()
n=10**5
p=317
while p < 340: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

347 733
CPU times: user 25.9 s, sys: 31.9 ms, total: 25.9 s
Wall time: 26.3 s


In [8]:
%%time
P=Primes()
n=10**5
p=347
while p < 360: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

353 811
359 12239
CPU times: user 10min 34s, sys: 1.47 s, total: 10min 35s
Wall time: 10min 48s


In [8]:
%%time
P=Primes()
n=10**5
p=359
result =[]
while p < 600:
    p=P.next(p)
    if p % 3 ==2:
        result.append(p)
print(result)    

[383, 389, 401, 419, 431, 443, 449, 461, 467, 479, 491, 503, 509, 521, 557, 563, 569, 587, 593, 599]
CPU times: user 0 ns, sys: 5.14 ms, total: 5.14 ms
Wall time: 9.37 ms


In [9]:
%%time 
m= 0
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

3000
383 -1
CPU times: user 2min 49s, sys: 326 ms, total: 2min 49s
Wall time: 2min 56s


In [10]:
%%time 
m= 3000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

6000
383 -1
CPU times: user 3min 4s, sys: 282 ms, total: 3min 4s
Wall time: 3min 15s


In [11]:
%%time 
m= 6000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

9000
383 -1
CPU times: user 3min 6s, sys: 334 ms, total: 3min 7s
Wall time: 3min 11s


In [12]:
%%time 
m= 9000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

12000
383 -1
CPU times: user 3min 16s, sys: 581 ms, total: 3min 16s
Wall time: 3min 28s


In [8]:
%%time 
m= 12000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

15000
367 -1
CPU times: user 2min 51s, sys: 231 ms, total: 2min 51s
Wall time: 2min 57s


In [13]:
%%time 
m= 15000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

18000
383 -1
CPU times: user 3min 33s, sys: 350 ms, total: 3min 34s
Wall time: 3min 50s


In [14]:
%%time 
m= 18000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

21000
383 -1
CPU times: user 3min 9s, sys: 330 ms, total: 3min 9s
Wall time: 3min 22s


In [3]:
%%time 
m= 21000
n= m+3000
print(n)
p=383
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

24000
383 23539
CPU times: user 2min 41s, sys: 219 ms, total: 2min 41s
Wall time: 2min 43s


In [4]:
%%time 
m= 0
n= m+3000
print(n)
p=389
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

3000
389 -1
CPU times: user 2min 41s, sys: 182 ms, total: 2min 41s
Wall time: 2min 45s


In [7]:
%%time 
m= 3000
n= m+3000
print(n)
p=389
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

6000
389 -1
CPU times: user 3min, sys: 182 ms, total: 3min 1s
Wall time: 3min 3s


In [8]:
%%time 
m= 6000
n= m+3000
print(n)
p=389
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

9000
389 6733
CPU times: user 52.5 s, sys: 151 ms, total: 52.7 s
Wall time: 53.4 s


In [9]:
%%time 
m= 0
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

3000
401 -1
CPU times: user 2min 57s, sys: 381 ms, total: 2min 57s
Wall time: 3min


In [10]:
%%time 
m= 3000
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

6000
401 -1
CPU times: user 3min 47s, sys: 238 ms, total: 3min 47s
Wall time: 3min 51s


In [12]:
%%time 
m= 6000
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

9000
401 -1
CPU times: user 3min 2s, sys: 167 ms, total: 3min 2s
Wall time: 3min 4s


In [13]:
%%time 
m= 9000
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

12000
401 -1
CPU times: user 3min 36s, sys: 333 ms, total: 3min 36s
Wall time: 3min 45s


In [14]:
%%time 
m= 12000
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

15000
401 -1
CPU times: user 3min 23s, sys: 226 ms, total: 3min 24s
Wall time: 3min 29s


In [15]:
%%time 
m= 15000
n= m+3000
print(n)
p=401
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

18000
401 17987
CPU times: user 3min 35s, sys: 313 ms, total: 3min 35s
Wall time: 3min 39s


In [16]:
%%time 
m= 0
n= m+3000
print(n)
p=419
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

3000
419 -1
CPU times: user 3min 21s, sys: 218 ms, total: 3min 21s
Wall time: 3min 30s


In [18]:
%%time 
m= 3000
n= m+3000
print(n)
p=419
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

6000
419 -1
CPU times: user 3min 18s, sys: 290 ms, total: 3min 19s
Wall time: 3min 21s


In [19]:
%%time 
m= 6000
n= m+3000
print(n)
p=419
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

9000
419 6871
CPU times: user 1min 20s, sys: 91.1 ms, total: 1min 21s
Wall time: 1min 25s


In [3]:
%%time 
m= 0
n= m+3000
print(n)
p=431
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

3000
431 -1
CPU times: user 3min 29s, sys: 206 ms, total: 3min 30s
Wall time: 3min 36s


In [3]:
%%time 
m= 0
n= 10**5
print(n)
p=431
f = fekete(3*p)
print(p, search_two_four_cycle_2(f,m,n))

100000
431 9859
CPU times: user 13min 27s, sys: 9.08 s, total: 13min 36s
Wall time: 14min 4s


In [2]:
%%time
P=Primes()
n=10**5
p=431
while p < 450: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

443 7127
449 4651
CPU times: user 15min 44s, sys: 6.29 s, total: 15min 50s
Wall time: 16min 19s


In [4]:
%%time
P=Primes()
n=10**5
p=449
while p < 500: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

461 13781
467 1483
479 853


In [5]:
%%time
P=Primes()
n=10**5
p=479
while p < 600: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

491 3011


503 15749


509 16937


521 13009


557 7013


563 8783


569 157


587 15319


593 19709


599 5647
CPU times: user 4h 19min 21s, sys: 1min 43s, total: 4h 21min 5s
Wall time: 4h 28min 20s


In [6]:
%%time
P=Primes()
n=10**5
p=599
while p < 700: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

617 3877


641 1523


647 13723


653 3329


659 27011


677 20123


683 10993


701 11657
CPU times: user 6h 3min 10s, sys: 2min 3s, total: 6h 5min 14s
Wall time: 6h 49min 47s


In [7]:
%%time
P=Primes()
n=10**5
p=701
while p < 800: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

719 9043


743 8543


761 87433


773 10247


797 35801


809 4733
CPU times: user 15h 46min 58s, sys: 12min 6s, total: 15h 59min 4s
Wall time: 18h 45min 49s


In [8]:
%%time
P=Primes()
n=10**5
p=809
while p < 900: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

821 3917


827 22189


839 1187


857 5437


863 4957


881 21467


887 2711
CPU times: user 7h 44min 22s, sys: 5min 41s, total: 7h 50min 4s
Wall time: 12h 17min 58s


In [9]:
%%time
P=Primes()
n=10**5
p=887
while p < 1000: 
    p=P.next(p)
    if p % 3 ==2: 
        f = fekete(3*p)
        print(p, search_two_four_cycle(f,n))

911 29191


929 2689


941 14929


947 22621


953 37463


971 15401


977 10889


983 443
CPU times: user 21h 30min 13s, sys: 9min 51s, total: 21h 40min 4s
Wall time: 1d 10h 14min 24s


# Example 

We discuss a concrete example in this case, namely the case $n=3 \times 7$

In [12]:
f=fekete(3*7)
f

x^16 + x^15 + x^9 + x^7 + x + 1

In [14]:
g=reduced_fekete(3*7)
g

x^8 + x^7 - 8*x^6 - 7*x^5 + 20*x^4 + 14*x^3 - 16*x^2 - 6*x + 2

In [15]:
fekete_reduction(g, 5)

x^8 + x^7 + 2*x^6 + 3*x^5 + 4*x^3 + 4*x^2 + 4*x + 2

In [16]:
fekete_reduction(g, 19)

(x + 8) * (x^7 + 12*x^6 + 10*x^5 + 8*x^4 + 13*x^3 + 5*x^2 + x + 5)

In [17]:
fekete_reduction(g, 7)

(x^2 + x + 4) * (x^3 + 4) * (x^3 + 2*x + 1)

In [18]:
fekete_reduction(f, 227)

(x^2 + 12*x + 1) * (x^7 + 78*x^6 + 173*x^5 + 18*x^4 + 119*x^3 + 129*x^2 + 107*x + 9) * (x^7 + 138*x^6 + 90*x^5 + 215*x^4 + 2*x^3 + 221*x^2 + 160*x + 101)